##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/embeddinggemma/fine-tuning-embeddinggemma-with-sentence-transformers"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/embeddinggemma/fine-tuning-embeddinggemma-with-sentence-transformers.ipynb""><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/google/generative-ai-docs/blob/main/site/en/gemma/docs/embeddinggemma/fine-tuning-embeddinggemma-with-sentence-transformers.ipynb"><img src="https://www.kaggle.com/static/images/logos/kaggle-logo-transparent-300.png" height="32" width="70"/>Run in Kaggle</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/google/generative-ai-docs/main/site/en/gemma/docs/embeddinggemma/fine-tuning-embeddinggemma-with-sentence-transformers.ipynb"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />Open in Vertex AI</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemma/docs/embeddinggemma/fine-tuning-embeddinggemma-with-sentence-transformers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Fine-tune EmbeddingGemma

Fine-tuning helps close the gap between a model's general-purpose understanding and the specialized, high-performance accuracy that your application requires. Since no single model is perfect for every task, fine-tuning adapts it to your specific domain.

Imagine your company, "Shibuya Financial" offers various complex financial products like investment trusts, NISA accounts (a tax-advantaged savings account), and home loans. Your customer support team uses an internal knowledge base to quickly find answers to customer questions.

## Setup

Before starting this tutorial, complete the following steps:

* Get access to EmbeddingGemma by logging into [Hugging Face](https://huggingface.co/google/embeddinggemma-300M) and selecting **Acknowledge license** for a Gemma model.
* Generate a Hugging Face [Access Token](https://huggingface.co/docs/hub/en/security-tokens#how-to-manage-user-access-token) and use it to login from Colab.

This notebook will run on either CPU or GPU.

### Install Python packages

Install the libraries required for running the EmbeddingGemma model and generating embeddings. Sentence Transformers is a Python framework for text and image embeddings. For more information, see the [Sentence Transformers](https://www.sbert.net/) documentation.

In [24]:
!pip install -U sentence-transformers git+https://github.com/huggingface/transformers@v4.56.0-Embedding-Gemma-preview

  Cloning https://github.com/huggingface/transformers (to revision v4.56.0-Embedding-Gemma-preview) to /tmp/pip-req-build-um6l9hty
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-um6l9hty
  Running command git checkout -q 60b68e304cf4b6569b0660a13b558b929d4b0e77
  Resolved https://github.com/huggingface/transformers to commit 60b68e304cf4b6569b0660a13b558b929d4b0e77
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


After you have accepted the license, you need a valid Hugging Face Token to access the model.

In [25]:
# Login into Hugging Face Hub
from huggingface_hub import login
login()

### Load Model

Use the `sentence-transformers` libraries to create an instance of a model class with EmbeddingGemma.

In [26]:
import torch
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"

model_id = "google/embeddinggemma-300M"
model = SentenceTransformer(model_id).to(device=device)

print(f"Device: {model.device}")
print(model)
print("Total number of parameters in the model:", sum([p.numel() for _, p in model.named_parameters()]))

Device: cuda:0
SentenceTransformer(
  (0): Transformer({'max_seq_length': 2048, 'do_lower_case': False, 'architecture': 'Gemma3TextModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 3072, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (3): Dense({'in_features': 3072, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (4): Normalize()
)
Total number of parameters in the model: 307581696


## Prepare the Fine-Tuning Dataset

This is the most crucial part. You need to create a dataset that teaches the model what "similar" means in your specific context. This data is often structured as triplets: (anchor, positive, negative)

- Anchor: The original query or sentence.
- Positive: A sentence that is semantically very similar or identical to the anchor.
- Negative: A sentence that is on a related topic but semantically distinct.

In this example, we only prepared 3 triplets, but for a real application, you would need a much larger dataset to perform well.

In [33]:
import json
import random
from datasets import Dataset

# import train_test_split

from sklearn.model_selection import train_test_split



# Load the JSON dataset from the file

with open('synthetic_pairs.json', 'r') as f:
  synthetic_data = json.load(f)





# Process the data to include the prompt instruction for the anchor and sample one negative

processed_data = []

for item in synthetic_data:
  anchor = item["query"]
  positive = item["pos"][0]
  # random choice neg
  negative = random.choice(item["neg"])
  processed_data.append({"query": anchor, "positive": positive, "negative": negative})

X, y = train_test_split(processed_data, test_size=0.2, random_state=43)

X = Dataset.from_list(X)

y = Dataset.from_list(y)



print(X)

print(y)

Dataset({
    features: ['query', 'positive', 'negative'],
    num_rows: 2790
})
Dataset({
    features: ['query', 'positive', 'negative'],
    num_rows: 698
})


## Before Fine-Tuning

A search for "tax-free investment" might have given the following results, with similarity scores:

1. Document: Opening a NISA account (Score: 0.51)
2. Document: Opening a Regular Saving Account (Score: 0.50) <- *Similar score, potentially confusing*
3. Document: Home Loan Application Guide (Score: 0.44)

> Note: To generate optimal embeddings with EmbeddingGemma, you should add an "instructional prompt" or "task" to the beginning of your input text. You will use `STS` for sentence similarity. For details on all available EmbeddingGemma prompts, see the [model card](http://ai.google.dev/gemma/docs/embeddinggemma/model_card#prompt_instructions).

In [34]:

get_scores(query, documents)

Document:  Opening a NISA Account -> 🤖 Score:  0.7175183
Document:  Opening a Regular Savings Account -> 🤖 Score:  0.3492574
Document:  Home Loan Application Guide -> 🤖 Score:  0.46203923


## Training

Using a framework like `sentence-transformers` in Python, the base model gradually learns the subtle distinctions in your financial vocabulary.

In [ ]:
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.losses import MultipleNegativesRankingLoss
from transformers import TrainerCallback
from sentence_transformers.evaluation import TripletEvaluator, RerankingEvaluator

task_name = "Retrieval-query"


loss = MultipleNegativesRankingLoss(model)

args = SentenceTransformerTrainingArguments(
# Required parameter:
output_dir="polymatch-gemma",
# Optional training parameters:
prompts=model.prompts[task_name], # use model's prompt to train
num_train_epochs=5,
per_device_train_batch_size=64,
learning_rate=2e-5,
warmup_ratio=0.1,
# Optional tracking/debugging parameters:
logging_steps=X.num_rows,
report_to="none",
)


dev_evaluator = RerankingEvaluator(X)


_eval = dev_evaluator(model)

print(_eval)

trainer = SentenceTransformerTrainer(
model=model,
args=args,
train_dataset=X,
loss=loss,
eval_dataset=y,
evaluator=dev_evaluator,
)





trainer.train()
test_evaluator = RerankingEvaluator(y)

_eval2 = test_evaluator(model)

print(_eval2)

## After Fine-Tuning

The same search now yields much clearer results:

1. Document: Opening a NISA account (Score: 0.73) <- *Much more confident*
2. Document: Opening a Regular Saving Account (Score: 0.34) <- *Clearly less relevant*
3. Document: Home Loan Application Guide (Score: 0.47)

To upload your model to the Hugging Face Hub, you can use the `push_to_hub` method from the Sentence Transformers library.

Uploading your model makes it easy to access for inference directly from the Hub, share with others, and version your work. Once uploaded, anyone can load your model with a single line of code, simply by referencing its unique model ID `<username>/my-embedding-gemma`


In [ ]:
# Push to Hub
model.push_to_hub("my-embedding-gemma")

## Summary and next steps

You have now learned how to adapt an EmbeddingGemma model for a specific domain by fine-tuning it with the Sentence Transformers library.

Explore what more you can do with EmbeddingGemma:

* [Training Overview](https://sbert.net/docs/sentence_transformer/training_overview.html) in Sentence Transformers Documentation
* [Generate embeddings with Sentence Transformers](https://ai.google.dev/gemma/docs/embeddinggemma/inference-embeddinggemma-with-sentence-transformers)
* [Simple RAG example](https://github.com/google-gemini/gemma-cookbook/blob/main/Gemma/%5BGemma_3%5DRAG_with_EmbeddingGemma.ipynb) in the Gemma Cookbook
